In [1]:
import numpy as np
import pandas as pd
from copy import deepcopy as dc

#### приведение к нормальной форме Фробениуса

In [2]:
def to_frob_nf(A, log=False):
    A = dc(A)
    N = len(A)
    Bs = np.eye(N)
    for k in range(N-1, 0, -1):
        B, Bi = np.eye(N), np.eye(N)
        B[k-1] = -A[k]/A[k,k-1]
        B[k-1,k-1] = 1/A[k, k-1]
        Bi[k-1] = A[k]
        A = Bi @ A @ B
        Bs = Bs @ B
        if log:
            print('-'*30)
            print('B')
            print(B)
            print('Bi')
            print(Bi)
            print('new A')
            print(A)

    return A, Bs

#### применение теоремы Гершгольма

In [3]:
def check_gersh(A):
    assert not (A - A.T).any()
    print('Проверка на симметричность пройдена')

In [4]:
class Interval:
    a: float
    b: float
    i: int

    def __init__(self, a, b, i) -> None:
        self.a, self.b ,self.i = a, b, i

    def __repr__(self):
        return f'({self.a}<-{self.i}->{self.b})'

    def len(self):
        return self.b - self.a
    
    def __add__(self, o):
        return Interval(
            min(self.a, o.a),
            max(self.b, o.b), 
            self.i + o.i)

    def __hash__(self) -> int:
        return self.i

    def inter(self, o):
        return self.a <= o.a <= self.b or o.a <= self.a <= o.b 
    
    @staticmethod
    def from_row(row, i):
        c = row[i]
        r = abs(row).sum() - abs(row[i])
        return Interval(c-r, c+r, 1)
    
    def __eq__(self, o) -> bool:
        return self.a == o.a and self.b == o.b and self.i == o.i


In [6]:
def find_gersh(A):
    N = len(A)
    intervals = set(Interval.from_row(A[i], i) for i in range(N))

    while True:
        to_remove = set()
        to_add = set()
        for x in intervals:
            for y in intervals - {x}:
                if x.inter(y):
                    to_remove |= {x,y}
                    to_add |= {x+y}
                    break
            else:
                continue
            break

        if to_remove == to_add == set():
            break

        intervals -= to_remove
        intervals |= to_add

    return intervals

#### построение полиномиальной функции

In [7]:
class Poly:
    def __init__(self, coefs) -> None:
        self.coefs = coefs[::-1]

    def __call__(self, x) -> float:
        return sum([ x**i * c for i, c in enumerate(self.coefs)])

#### поиск нулей функции на интервале

In [8]:
def lin_search(p, a, b, delta = 0.05):
    linsp = np.arange(a, b+delta, delta)
    search_here = []
    for ai, bi in zip(linsp, linsp[1:]):
        if p(ai) * p(bi) < 0:
            search_here.append((ai, bi))

    return search_here

def binary_search(p, a, b, delta = 0.001):
    c = (a+b)/2
    if b-a < delta:
        return c
    if p(c) == 0:
        return c
    try:
        if p(a)*p(c) < 0:
            return binary_search(p, a, c, delta)
        return binary_search(p, c, b, delta)
    except RecursionError:
        return c

#### метод Крылова

In [9]:
def krylov_find_ps(A):
    N = len(A)

    # генерируем y_k - е
    y0 = np.zeros(N)
    y0[0] = 1
    Yks = [y0]
    # Yks = [np.random.rand(N)]
    for _ in range(N-1):
        Yks.append(A@Yks[-1])

    # составляем СЛАУ для нахождения p_i
    b = A@Yks[-1]
    Ymat = np.array(dc(Yks[::-1])).T
    
    # находим p_i
    ps = np.linalg.solve(Ymat, b)
    return ps, np.array(Yks[::-1])

In [10]:
def krylov_eigenvecs(A, Yks, ps, lambdas):
    N = len(A)

    # составляем матрицу кожффициентов многочленов Q_i
    Q = np.empty((N,N))
    for i in range(N):
        for j in range(N):
            Q[j,i] = 1 if j == 0 else (lambdas[i]*Q[j-1,i] + ps[j-1])

    eigenvecs = []
    for i in range(N):
        x = np.zeros(N)
        for j in range(N):
            x += Q[j,i]*Yks[j]
        eigenvecs.append(x/np.linalg.norm(x))
    
    return eigenvecs


#### проверки результатов

In [11]:
def find_eigenvector(B, alpha):
    vec = np.array([alpha**i for i in range(len(B))][::-1])
    eigv = B.dot(vec)
    return eigv / np.linalg.norm(eigv)

def check_orthogonality(vecs):
    A = np.array([[v1@v2 for v2 in vecs] for v1 in vecs])
    print('ОРТОГОНАЛЬНОСТЬ:')
    print(A)

def check_eigenvalues(A, eigenvalues):
    print('|tr(A) - sum(eigenvalues)| =', abs(sum(eigenvalues) - np.trace(A)))

#### все вместе

In [12]:
DANYLEVSKY = 1
KRYLOV = 2

In [13]:
def main(TestMat, method=DANYLEVSKY):
    print(f'method: {method}')
    # проверяем симметричность
    check_gersh(TestMat)

    # ищемм коэффициенты характерристического многочлена
    if method == DANYLEVSKY:
        P, B = to_frob_nf(TestMat)
        P = P[0]
    elif method == KRYLOV:
        P, Yks = krylov_find_ps(TestMat)
    else: 
        raise RuntimeError(f'unknown method {method}')

    # строим хар. мн-н
    p = Poly([1] + list(-P))
    print(p.coefs)

    # ищем интервалы Гершгорина
    intervals = find_gersh(TestMat)

    # ищем нули полинома
    eigenvalues = []
    for inter in intervals:
        eigenvalues += [binary_search(p, *sh) for sh in lin_search(p, inter.a, inter.b)]


    print('СОБСТВЕННЫЕ ЗНАЧЕНИЯ:\n', eigenvalues)
    print()
    check_eigenvalues(TestMat, eigenvalues)
    print()

    if method == DANYLEVSKY:
        eigenvectors = np.array([find_eigenvector(B, alpha) for alpha in eigenvalues ])
    elif method == KRYLOV:
        eigenvectors = np.array(krylov_eigenvecs(TestMat, Yks, -P, eigenvalues))
    
    
    print('СОБСТВЕННЫЕ ВЕКТОРЫ:\n', eigenvectors)
    check_orthogonality(eigenvectors)

    return eigenvalues

### Тестирование

In [14]:
TestMat = np.array([2.2, 1, 0.5, 2, 1, 1.3, 2, 1, 0.5, 2, 0.5, 1.6, 2, 1, 1.6, 2]).reshape((4,4))

In [17]:
main(TestMat, method=DANYLEVSKY)

method: 1
Проверка на симметричность пройдена
[-2.7616000000000085, 12.735000000000014, -0.20000000000000284, -6.000000000000001, 1]
СОБСТВЕННЫЕ ЗНАЧЕНИЯ:
 [-1.419921875000007, 0.22226562499998678, 1.545703124999982, 5.651953124999968]

|tr(A) - sum(eigenvalues)| = 7.016609515630989e-14

СОБСТВЕННЫЕ ВЕКТОРЫ:
 [[-0.22223691  0.51596093 -0.75715654  0.33333023]
 [-0.52240375 -0.45437229  0.15367346  0.70499973]
 [ 0.62896892 -0.57252084 -0.48568631  0.20180878]
 [ 0.53214978  0.44599829  0.40845218  0.59251071]]
ОРТОГОНАЛЬНОСТЬ:
[[ 1.00000000e+00  3.01909159e-04 -1.68954764e-04  9.38629489e-05]
 [ 3.01909159e-04  1.00000000e+00 -8.00077296e-04 -1.58154035e-04]
 [-1.68954764e-04 -8.00077296e-04  1.00000000e+00  5.56586533e-04]
 [ 9.38629489e-05 -1.58154035e-04  5.56586533e-04  1.00000000e+00]]


[-1.419921875000007, 0.22226562499998678, 1.545703124999982, 5.651953124999968]

##### произвольной размерности

In [19]:
random_mat = np.random.rand(10,10)
random_mat += random_mat.T
random_mat
main(random_mat, method=DANYLEVSKY)

method: 1
Проверка на симметричность пройдена
[5.771302755360793, -36.510673115142595, 33.38900960339625, 105.96874301626735, -116.89820231567161, -112.51970374353232, 96.54109668878169, 55.68996077540049, -21.772270584104305, -8.530691325656404, 1]
СОБСТВЕННЫЕ ЗНАЧЕНИЯ:
 [-2.07181546175652, -1.7265029617565149, -1.243690461756508, -0.8413467117565024, 0.23443453824351287, 0.4758407882435163, 0.7563095382435203, 0.9274032882435227, 1.9586532882435375, 10.061778288243653]

|tr(A) - sum(eigenvalues)| = 0.00037280677874207413

СОБСТВЕННЫЕ ВЕКТОРЫ:
 [[-1.02236469e-01 -8.24840342e-02  4.77043316e-01 -3.06719684e-01
  -2.64143329e-01 -4.00127670e-01  2.27954725e-01 -2.29599124e-01
   5.30210709e-01  2.13120713e-01]
 [-1.58118531e-02 -6.51175117e-01  6.49743311e-02  4.69704180e-01
   2.76045090e-01  6.58608795e-02 -2.39993530e-01 -3.76383082e-01
   2.64266419e-01  3.52259096e-02]
 [ 9.12988390e-02  3.20559181e-01 -6.02541519e-01  3.33979482e-01
  -3.17001597e-01 -3.11356730e-02  2.15746214e-0

[-2.07181546175652,
 -1.7265029617565149,
 -1.243690461756508,
 -0.8413467117565024,
 0.23443453824351287,
 0.4758407882435163,
 0.7563095382435203,
 0.9274032882435227,
 1.9586532882435375,
 10.061778288243653]